In [1]:
import json
#import pydevd
#pydevd.settrace('localhost', port=49309, stdoutToServer=True, stderrToServer=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from six.moves import xrange
#from __pynauty__ import graph
#import pynauty.graph
import pynauty
import time
import tensorflow as tf
import networkx as nx
import numpy as np
import pynauty as nauty
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import os
import sys
sys.path.append('PatchyTools')
sys.path.append('../PatchyCapsules/')
from utils_caps import load_image_data
import tensorflow as tf
from numpy import random as nprand

from CapsuleNetwork import CapsuleNetwork
from utils_caps import load_image_data,subsample
from DropboxLoader import DropboxLoader
%matplotlib inline

from GraphConverter import GraphConverter
from PatchyTools import utils
from PatchyTools import Patchy_san
from Patchy_san import dfadj_to_dict
from Patchy_san import canonical_labeling

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data

In [2]:
# Path to data:
train_file_path = '../../others/CIFAR10-img-classification-tensorflow/cifar-10-batches-py/data_batch_1'
test_file_path = '../../others/CIFAR10-img-classification-tensorflow/cifar-10-batches-py/test_batchs'
X_train, y_train = load_image_data(train_file_path)
X_test, y_test = load_image_data(train_file_path)
X_train,y_train = subsample(X_train, y_train)
X_test, y_test = subsample(X_test, y_test)

# Constants:
batch_size = 100
height, width, num_channels = X_train.shape[1:]
num_inputs = height * width
num_outputs = 10
num_iter = 5
caps = CapsuleNetwork(batch_size,num_inputs,num_outputs,num_iter)

shape :  (10000, 32, 32, 3)
shape :  (10000, 32, 32, 3)


In [3]:
print('TensorFlow Version: {}'.format(tf.__version__)) 
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.6.0
Default GPU Device: 


In [4]:

'''
#Node id is made to start from 0 due to nauty package requirement, even if it starts from 1 in the original
#Graph id is starting from 1
'''


mutag = DropboxLoader('MUTAG')

df_edge_label = mutag.get_edge_label()
df_graph_ind = mutag.get_graph_ind()
df_adj = mutag.get_adj()
# Getting the
df_node_label = mutag.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)


In [5]:
from Patchy_san import create_adj_dict_by_graphId
from Patchy_san import get_smallest_node_id_from_adj
from Patchy_san import get_subset_adj
from Patchy_san import create_adj_coomatrix_by_graphId
from Patchy_san import canonical_labeling
from Patchy_san import make_neighbor
import utils

In [6]:
# Adjeceny dict and 
adj_dict = create_adj_dict_by_graphId(df_adj, df_node_label)
adj_coomatrix_by_graphId = create_adj_coomatrix_by_graphId(adj_dict, df_node_label)
cano_label = canonical_labeling(adj_dict, df_node_label, df_adj)

In [7]:
feature_list = df_node_label['label'].unique()
nodes_features = pd.get_dummies(df_node_label.label,
                                columns=feature_list,
                                sparse=True)
nodes_features = nodes_features.T.reindex(feature_list).T.fillna(0)

In [8]:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
PatchyConverter = GraphConverter(dataset_name, width, receptive_field)
mutag_tensor = PatchyConverter.graphs_to_Patchy_tensor()

In [9]:
g1 = nx.Graph()
g1.add_edges_from(adj_1.values)
nx.draw(g1)

Create dictionary of graphs
Canonical Labeling
Getting the Neighboors 
Neighboors to Tensor


In [11]:
mutag_tensor.shape

(188, 180, 7)

In [ ]:
g1 = nx.Graph()
g1.add_edges_from(adj_dict[1].values)
nx.draw(g1,with_labels=True)

In [ ]:
nauty_graph = nauty.Graph(len(temp_graph_dict), adjacency_dict=temp_graph_dict)
test_canon_label = nauty.canonical_labeling(nauty_graph)

In [ ]:
test_2 = [df_subset_nodes.label.values[i] for i in test_canon_label]  ###

In [ ]:
test_canon_label

In [ ]:
nodes_features.head()

In [ ]:
test_2

In [ ]:
df_node_label2 = pd.concat([df_node_label, pd.Series(cano_label, dtype=int, name='cano_label')], axis=1)


In [ ]:
unique_graph_labels = df_node_label.graph_ind.unique()


In [ ]:
adjacency = adj_coomatrix_by_graphId[1]
graph = nx.from_numpy_matrix(adjacency.todense())

# Create the neighbors with -1 for neighbor assemble.
#After this, if the RECEPTIVE_FIELD_SIZE_K exceeds the number of WIDTH_W, then fill them with -1
neighborhoods = np.zeros((18, 10), dtype=np.int32)
neighborhoods.fill(-1)

df_sequence = df_node_label[df_node_label.graph_ind == 1]
df_sequence = df_sequence.sort_values(by='cano_label')
smallest_node_id = df_sequence.node.min()

# CUT GRAPH BY THRESHOLD of cano_label ''' Top width w elements of V according to labeling  '''
df_sequence = df_sequence.iloc[:18,:]
df_sequence['node'] = df_sequence.node.values  - smallest_node_id


In [ ]:
df_sequence

In [ ]:
df_shortest = pd.DataFrame.from_dict(nx.single_source_dijkstra_path_length(graph, 8),orient='index') #

In [ ]:
RECEPTIVE_FIELD_SIZE_K =10

In [ ]:
df_shortest.columns =['distance'] #
df_shortest['node'] = df_shortest.index.values #
df_shortest = pd.merge(df_node_label, df_shortest, on='node', how='right') #

In [ ]:
# Sort by distance and then by cano_label
df_shortest = df_shortest.sort_values(by=['distance','cano_label']) #
df_shortest = df_shortest.iloc[:10,:] #

In [ ]:
for j in range(0, min(RECEPTIVE_FIELD_SIZE_K, len(df_shortest))):
    #neighborhoods[i][j] = shortest[j][0]
    neighborhoods[8][j] = df_shortest['node'].values[j] + smallest_node_id


In [ ]:
df_sequence


In [ ]:
WIDTH_W = 18

In [ ]:
unique_graph_labels = df_node_label.graph_ind.unique()
for l_ind, l in zip([0],[1]):#enumerate(unique_graph_labels):
    adjacency = adj_coomatrix_by_graphId[l]
    graph = nx.from_numpy_matrix(adjacency.todense())

    # Create the neighbors with -1 for neighbor assemble.
    #After this, if the RECEPTIVE_FIELD_SIZE_K exceeds the number of WIDTH_W, then fill them with -1
    neighborhoods = np.zeros((WIDTH_W, RECEPTIVE_FIELD_SIZE_K), dtype=np.int32)
    neighborhoods.fill(-1)

    df_sequence = df_node_label[df_node_label.graph_ind == l]
    df_sequence = df_sequence.sort_values(by='cano_label')
    smallest_node_id = df_sequence.node.min()

    # CUT GRAPH BY THRESHOLD of cano_label ''' Top width w elements of V according to labeling  '''
    df_sequence = df_sequence.iloc[:WIDTH_W,:]
    df_sequence['node'] = df_sequence.node.values  - smallest_node_id

    for i, node in enumerate(df_sequence.node):
        #shortest = nx.single_source_dijkstra_path_length(graph, node).items()
        df_shortest = pd.DataFrame.from_dict(nx.single_source_dijkstra_path_length(graph, node),
                                             orient='index') #
        df_shortest.columns =['distance'] #
        df_shortest['node'] = df_shortest.index.values #
        df_shortest = pd.merge(df_node_label, df_shortest, on='node', how='right') #

        # Sort by distance and then by cano_label
        df_shortest = df_shortest.sort_values(by=['distance','cano_label']) #
        df_shortest = df_shortest.iloc[:RECEPTIVE_FIELD_SIZE_K,:] #
        #shortest = sorted(shortest, key=lambda v: v[1])
        #shortest = shortest[:RECEPTIVE_FIELD_SIZE_K]
        for j in range(0, min(RECEPTIVE_FIELD_SIZE_K, len(df_shortest))):
            #neighborhoods[i][j] = shortest[j][0]
            neighborhoods[i][j] = df_shortest['node'].values[j] + smallest_node_id

    #neighborhoods_dict[l]= neighborhoods.copy()
    if l_ind ==0:
        neighborhoods_all = neighborhoods
    else:
        neighborhoods_all = np.r_[neighborhoods_all, neighborhoods]

In [ ]:
# Getting the adj matrix and the nodes:
df_subset_adj = adj_dict[1]
df_subset_nodes = df_node_label[df_node_label.graph_ind==1]

In [ ]:
temp_graph_dict = dfadj_to_dict(df_subset_adj)
temp_graph_dict

In [ ]:
adj_dict_by_graphId = create_adj_dict_by_graphId(df_adj, df_node_label)
adj_coomatrix_by_graphId = create_adj_coomatrix_by_graphId(adj_dict_by_graphId, df_node_label)

In [ ]:
df_node_label = pd.concat([df_node_label, pd.Series(cano_label, dtype=int, name='cano_label')], axis=1)

In [ ]:
neighborhoods_graph = make_neighbor(adj_coomatrix_by_graphId, df_node_label, WIDTH_W=18, RECEPTIVE_FIELD_SIZE_K=10)

In [ ]:
neighborhoods_graph

In [ ]:
len(df_node_label.graph_ind.unique())

In [ ]:
df_node_label.graph_ind.unique()

In [ ]:



df_node_label = pd.concat([df_node_label, pd.Series(cano_label, dtype=int, name='cano_label')], axis=1)

neighborhoods_graph = make_neighbor(adj_coomatrix_by_graphId, df_node_label, WIDTH_W=WIDTH_W, RECEPTIVE_FIELD_SIZE_K=RECEPTIVE_FIELD_SIZE_K)
result_tensor = tensor(neighborhoods_graph, WIDTH_W, RECEPTIVE_FIELD_SIZE_K, df_node_label)

In [ ]:
idx_ranges = np.cumsum([0] + df_graph_ind.groupby('graph_ind').count().node.tolist())

In [ ]:
graph_idx = 4
list_nodes_per_graph = list(range(idx_ranges[graph_idx],idx_ranges[graph_idx+1]))
adj_1 = df_adj[df_adj['from'].isin(list_nodes_per_graph) | df_adj['to'].isin(list_nodes_per_graph)]

In [ ]:
g1 = nx.Graph()
g1.add_edges_from(adj_1.values)
nx.draw(g1)

In [ ]:
num_features = len(df_node_label.label.unique())
print('number of features : {}'.format(num_features))
data_in_patchysan = Patchy_san.main(WIDTH_W=18, RECEPTIVE_FIELD_SIZE_K=10, datasetname='MUTAG')
data_in_patchysan.shape[3]
#final_labels = mutag.get_graph_label().graph_label.values
#final_labels = pd.get_dummies(final_labels).values

In [ ]:
dataset_name = 'MUTAG'
GAMMA_ENV = os.environ['GAMMA_DATA_ROOT']
root_gamma_path = GAMMA_ENV+'Samples'
node_label_filename =  '{0}/{0}_node_labels.txt'.format(dataset_name)
node_label_path = os.path.join(root_gamma_path, node_label_filename)
pd.read_csv(node_label_path , delimiter=' ',header=None,index_col=None)

### Functions

In [ ]:

def get_subset_adj(df_adj, df_node_label,graph_label_num):
    df_glabel = df_node_label[df_node_label.graph_label == graph_label_num ]
    index_of_glabel = (df_adj['to'].isin(df_glabel.node) & df_adj['from'].isin(df_glabel.node))
    return df_adj[index_of_glabel]

def get_smallest_node_id_from_adj(df_adj):
    return min(df_adj['to'].min(), df_adj['from'].min())


def create_adj_dict_by_graphId(df_adj, df_node_label):
    '''
    input: df_node_label
    return: {1: {0:[0,2,5]}} = {graphId: {nodeId:[node,node,node]}}
    '''
    adj_dict_by_graphId ={}
    unique_graph_labels = df_node_label.graph_label.unique()
    for l in unique_graph_labels:
        df_subset_adj = get_subset_adj(df_adj, df_node_label, graph_label_num=l)        
        smallest_node_id = get_smallest_node_id_from_adj(df_subset_adj)
        df_subset_adj -= smallest_node_id
        adj_dict_by_graphId[l] = df_subset_adj
    return adj_dict_by_graphId


def canonical_labeling(adj_dict_by_graphId, df_node_label, df_adj):
    all_canonical_labels =[]
    unique_graph_labels = df_node_label.graph_label.unique()
    for l in unique_graph_labels:
        df_subset_adj = adj_dict_by_graphId[l]
        df_subset_nodes = df_node_label[df_node_label.graph_label==l]        
        temp_graph_dict = utils.dfadj_to_dict(df_subset_adj)
        nauty_graph = nauty.Graph(len(temp_graph_dict), adjacency_dict=temp_graph_dict)
        canonical_labeling = nauty.canonical_labeling(nauty_graph)        
        canonical_labeling = [df_subset_nodes.label.values[i] for i in canonical_labeling] ###
        all_canonical_labels += canonical_labeling
    return all_canonical_labels


def create_adj_coomatrix_by_graphId(adj_dict_by_graphId, df_node_label):
    """
    return: a coomatrix per graphId
    """
    
    adj_coomatrix_by_graphId ={}
    unique_graph_labels = df_node_label.graph_label.unique()
    for l in unique_graph_labels:
        df_subset_adj = adj_dict_by_graphId[l]
        df_subset_node_label = df_node_label[df_node_label.graph_label == l]
        adjacency = coo_matrix(( np.ones(len(df_subset_adj)), 
                                (df_subset_adj.iloc[:,0].values, df_subset_adj.iloc[:,1].values) ), 
                                 shape=(len(df_subset_node_label), len(df_subset_node_label))
                              )
        adj_coomatrix_by_graphId[l]=adjacency
    return adj_coomatrix_by_graphId

def make_neighbor(adj_coomatrix_by_graphId, df_node_label, WIDTH_W):
    
    """
    return: a dictionary with the shape of {graphId:[matrix: node x neighbor]} 
    The size of 2D matrix is (Node number) x (RECEPTIVE_FIELD_SIZE_K). 
    """
    
    neighborhoods_dict=dict()
    
    unique_graph_labels = df_node_label.graph_label.unique()
    for l in unique_graph_labels:
        adjacency = adj_coomatrix_by_graphId[l]
        graph = nx.from_numpy_matrix(adjacency.todense())
        
        # Create the neighbors with -1 for neighbor assemble.
        #After this, if the RECEPTIVE_FIELD_SIZE_K exceeds the number of WIDTH_W, then fill them with -1
        neighborhoods = np.zeros((WIDTH_W, RECEPTIVE_FIELD_SIZE_K), dtype=np.int32)
        neighborhoods.fill(-1) 
        
        df_sequence = df_node_label[df_node_label.graph_label == l]
        df_sequence = df_sequence.sort_values(by='cano_label')
        smallest_node_id = df_sequence.node.min()
                
        # CUT GRAPH BY THRESHOLD of cano_label ''' Top width w elements of V according to labeling  '''
        df_sequence = df_sequence.iloc[:WIDTH_W,:]
        df_sequence['node'] = df_sequence.node.values  - smallest_node_id        
        
        for i, node in enumerate(df_sequence.node):
            #shortest = nx.single_source_dijkstra_path_length(graph, node).items()
            df_shortest = pd.DataFrame.from_dict(nx.single_source_dijkstra_path_length(graph, node),
                                                 orient='index') #
            df_shortest.columns =['distance'] #
            df_shortest['node'] = df_shortest.index.values #
            df_shortest = pd.merge(df_node_label, df_shortest, on='node', how='right') #
            
            # Sort by distance and then by cano_label            
            df_shortest = df_shortest.sort_values(by=['distance','cano_label']) #
            df_shortest = df_shortest.iloc[:RECEPTIVE_FIELD_SIZE_K,:] #
            #shortest = sorted(shortest, key=lambda v: v[1])            
            #shortest = shortest[:RECEPTIVE_FIELD_SIZE_K]
            for j in range(0, min(RECEPTIVE_FIELD_SIZE_K, len(df_shortest))):
                #neighborhoods[i][j] = shortest[j][0]
                neighborhoods[i][j] = df_shortest['node'].values[j]
                
        neighborhoods_dict[l]= neighborhoods.copy()
    return neighborhoods_dict




### Arguments  

In [ ]:

now = time.time()

#NUM_NODES 
LABEL_THRESHOLD = 2 #threshold of canonical label
RECEPTIVE_FIELD_SIZE_K = 20 #''' Receptive Field Size'''
WIDTH_W = 8

# Main (Timing starts here)

In [ ]:
adj_dict_by_graphId = create_adj_dict_by_graphId(df_adj, df_node_label)
cano_label = canonical_labeling(adj_dict_by_graphId, df_node_label, df_adj)
df_node_label = pd.concat([df_node_label, pd.Series(cano_label, dtype=int, name='cano_label')],  axis=1)

#cert_list = [i for i in (nauty.certificate(nauty_graph))]
# '''canonical_labeling = [df_node_label.label.values[i] for i in canonical_labeling]'''

###### Show the frequency of labels to make threshold

In [ ]:
'''
# #How to select top w elements of V according to labeling  
df_node_label.cano_label.value_counts().plot(kind='bar')
df_node_label.cano_label.value_counts().sort_index().plot(kind='bar',  figsize=(14,5))
plt.title('Number of nodes by labeling')
plt.xlabel('Labeling')
plt.ylabel('Number of nodes')

_SUM_ALL_NODES = df_node_label.shape[0]
plt.twinx()
plt.ylabel("Cummlative Sum Rate", color="r")
plt.tick_params(axis="y", labelcolor="r")
plt.plot(df_node_label.cano_label.value_counts().sort_index().index, 
         df_node_label.cano_label.value_counts().sort_index().cumsum() /_SUM_ALL_NODES, "r-", linewidth=2)
plt.show()
'''

### Get several nodes with a condition of cano_label (sequence)

In [ ]:
adj_coomatrix_by_graphId = create_adj_coomatrix_by_graphId(adj_dict_by_graphId, df_node_label)
neighborhoods_graph = make_neighbor(adj_coomatrix_by_graphId, df_node_label, WIDTH_W=WIDTH_W)

### Things about tensorflow constraction

In [ ]:
"""
neighborhoods[graphId]: This represents the matrix of (nodes x neighbor).
nodes: This represents the matrix of (nodes x features).
"""

In [ ]:
feature_list = df_node_label['label'].unique()
num_features = len(feature_list)

def main_timing(graph_id):
    neighborhoods = tf.constant(neighborhoods_graph[graph_id], dtype=tf.int32)
    sparse_df = pd.get_dummies(df_node_label.loc[df_node_label.graph_label==graph_id].label, 
                               columns=feature_list,
                               sparse=True
                              )
    
    #### Reindex and transporse to get columns of get dummy #########
    sparse_df = sparse_df.T.reindex(feature_list).T.fillna(0)
    nodes = tf.constant(sparse_df.values, dtype=tf.int32 )
    
    with tf.Session() as sess:
        data = tf.reshape(neighborhoods, [-1])
        i = tf.maximum(data, 0)
        i_list = i.eval()
        #print(i_list)
        #print('')

        for ind, i in enumerate(i_list):
            if ind ==0: 
                positive = tf.strided_slice(nodes, [i], [i+1], [1])
                #positive = tf.constant(temp)
            else:
                temp = tf.strided_slice(nodes, [i], [i+1], [1])
                positive = tf.concat([positive,temp],axis=0)

        negative = tf.zeros([positive.shape[0], positive.shape[1]], dtype= tf.int32)

        #print ('shape', data.shape, negative.shape, positive.shape)
        #print ( data, negative, positive)

        # padding with 0 here because -1 indicates there is no neighbor nodes.
        ret = tf.where(data < 0, negative, positive)
        #print('padding done')


        ret = tf.reshape(ret, 
                         [neighborhoods.shape[0], 
                          RECEPTIVE_FIELD_SIZE_K, # This is equals to neighborhoods.shape[1]
                          num_features])
        #print (key, 'ret.shape: ',ret.shape)


# Timing

In [ ]:
#stop stp stp
for key in tqdm(adj_dict_by_graphId.keys()):
    main_timing(key)

print ('time passed in seconds', ("%.2f"%(time.time() - now)))